In [5]:
import os
import numpy as np
import pandas as pd
import time
from tqdm import tqdm

In [6]:
PATH = "../../data/olist/"

In [7]:
%%time
df_orders = pd.read_parquet(f'{PATH}olist_orders_dataset.parquet')
df_customers = pd.read_parquet(f'{PATH}olist_customers_dataset.parquet')
df_order_items = pd.read_parquet(f'{PATH}olist_order_items_dataset.parquet')
df_products = pd.read_parquet(f'{PATH}olist_products_dataset.parquet')
df_sellers = pd.read_parquet(f'{PATH}olist_sellers_dataset.parquet')

CPU times: total: 469 ms
Wall time: 308 ms


# PivotTable

In [9]:
# few variables and few agg function
list_product_id=[
    '4244733e06e7ecb4970a6e2683c13e61',
    'e5f2d52b802189ee658865ca93d83a8f',
    'c777355d18b72b67abbeef9df44fd0fd',
    'ac6c3623068f30de03045865e4e10089'
]
df_pivot_table=pd.pivot_table(
    df_order_items[df_order_items['product_id'].isin(list_product_id)], # source dataset
    values=['price','freight_value'], 
    index=['product_id','order_item_id'], 
    columns=['seller_id'], 
    aggfunc={
        'price':[np.sum, max],
        'freight_value':[np.mean, min],
            }, 
    fill_value=0, 
    margins=False, 
    )
df_pivot_table 

freight_value  \
                                                                           mean   
seller_id                                      48436dade18ac8b2bce089ec2a041202   
product_id                       order_item_id                                    
4244733e06e7ecb4970a6e2683c13e61 1                                      22.5225   
                                 2                                      18.1200   
ac6c3623068f30de03045865e4e10089 1                                       0.0000   
c777355d18b72b67abbeef9df44fd0fd 1                                       0.0000   
                                 2                                       0.0000   
e5f2d52b802189ee658865ca93d83a8f 1                                       0.0000   

                                                                                 \
                                                                                  
seller_id                                      5b51032eddd242adc84c38acab88f23d   
product_id                       order_item_id                                    
4244733e06e7ecb4970a6e2683c13e61 1                                         0.00   
                                 2                                         0.00   
ac6c3623068f30de03045865e4e10089 1                                         0.00   
c777355d18b72b67abbeef9df44fd0fd 1                                        17.87   
                                 2                                        20.08   
e5f2d52b802189ee658865ca93d83a8f 1                                         0.00   

                                                                                 \
                                                                                  
seller_id                                      dd7ddc04e1b6c2c614352b383efe2d36   
product_id                       order_item_id                                    
4244733e06e7ecb4970a6e2683c13e61 1                                         0.00   
                                 2                                         0.00   
ac6c3623068f30de03045865e4e10089 1                                         0.00   
c777355d18b72b67abbeef9df44fd0fd 1                                         0.00   
                                 2                                         0.00   
e5f2d52b802189ee658865ca93d83a8f 1                                        19.93   

                                                                                 \
                                                                                  
seller_id                                      df560393f3a51e74553ab94004ba5c87   
product_id                       order_item_id                                    
4244733e06e7ecb4970a6e2683c13e61 1                                       0.0000   
                                 2                                       0.0000   
ac6c3623068f30de03045865e4e10089 1                                      27.0525   
c777355d18b72b67abbeef9df44fd0fd 1                                       0.0000   
                                 2                                       0.0000   
e5f2d52b802189ee658865ca93d83a8f 1                                       0.0000   

                                                                                 \
                                                                            min   
seller_id                                      48436dade18ac8b2bce089ec2a041202   
product_id                       order_item_id                                    
4244733e06e7ecb4970a6e2683c13e61 1                                        13.29   
                                 2                                        18.12   
ac6c3623068f30de03045865e4e10089 1                                         0.00   
c777355d18b72b67abbeef9df44fd0fd 1                                         0.00   
                                 2                                         0.0

# Query

In [10]:
df_order_items.groupby(['seller_id','product_id'])['price'].mean().reset_index().query("seller_id=='001cca7ae9ae17fb1caed9dfb1094831'").head()

,seller_id,product_id,price
1,001cca7ae9ae17fb1caed9dfb1094831,08574b074924071f4e201e151b152b4e,95.215044
2,001cca7ae9ae17fb1caed9dfb1094831,0da9ffd92214425d880de3f94e74ce39,107.054706
3,001cca7ae9ae17fb1caed9dfb1094831,21fecd254a3103704126b28478ea7980,112.333333
4,001cca7ae9ae17fb1caed9dfb1094831,4d7fee7877228c1497477ae53d97c214,89.000000
5,001cca7ae9ae17fb1caed9dfb1094831,4f3b83b83f7fb280fca4f90ba502f50e,99.500000


In [11]:
list_sellers=['001cca7ae9ae17fb1caed9dfb1094831','ffff564a4f9085cd26170f4732393726']
df_order_items.groupby(['seller_id','product_id'])['price'].mean().reset_index().query("seller_id in @list_sellers").head()

,seller_id,product_id,price
1,001cca7ae9ae17fb1caed9dfb1094831,08574b074924071f4e201e151b152b4e,95.215044
2,001cca7ae9ae17fb1caed9dfb1094831,0da9ffd92214425d880de3f94e74ce39,107.054706
3,001cca7ae9ae17fb1caed9dfb1094831,21fecd254a3103704126b28478ea7980,112.333333
4,001cca7ae9ae17fb1caed9dfb1094831,4d7fee7877228c1497477ae53d97c214,89.000000
5,001cca7ae9ae17fb1caed9dfb1094831,4f3b83b83f7fb280fca4f90ba502f50e,99.500000


In [12]:
list_sellers=['ffff564a4f9085cd26170f4732393726']
list_products=['21fecd254a3103704126b28478ea7980']
df_order_items.groupby(['seller_id','product_id'])['price'].mean().reset_index().query("seller_id in @list_sellers or product_id in @list_products").head()

,seller_id,product_id,price
3,001cca7ae9ae17fb1caed9dfb1094831,21fecd254a3103704126b28478ea7980,112.333333
34433,ffff564a4f9085cd26170f4732393726,1b34988d6060d833cbe3bf2bdeeb67ab,366.500000
34434,ffff564a4f9085cd26170f4732393726,24905035adec59d4ab5a46c13973ef17,47.500000
34435,ffff564a4f9085cd26170f4732393726,47dbce97622eb7520b80d26ab613414e,51.450000
34436,ffff564a4f9085cd26170f4732393726,5b9d4c71cc2d0a2417c2635818b654d7,82.950000


# GROUP BY

In [13]:
df_order_items.groupby(['seller_id','product_id'])[['price','freight_value']].agg(['min', 'max'])

price  \
                                                                      min   
seller_id                        product_id                                 
0015a82c2db000af6aaaf3ae2ecb0532 a2ff5a97bf95719e38ea2e3b4105bce8  895.00   
001cca7ae9ae17fb1caed9dfb1094831 08574b074924071f4e201e151b152b4e   69.90   
                                 0da9ffd92214425d880de3f94e74ce39   99.99   
                                 21fecd254a3103704126b28478ea7980   99.00   
                                 4d7fee7877228c1497477ae53d97c214   89.00   
...                                                                   ...   
ffff564a4f9085cd26170f4732393726 cb17b798ba889575385281b7ec2fc22b  109.20   
                                 dbd024d4182504993ad1e3cd2ee9d9e9   29.40   
                                 de6517dda8e49774f58c07f80abc8d7a   69.00   
                                 e20b58fe57d487f33247e6cc1154eb9c  103.95   
                                 fa2de9e6a52f697fbd74ddbc2c15b16c  114.50   

                                                                           \
                                                                      max   
seller_id                        product_id                                 
0015a82c2db000af6aaaf3ae2ecb0532 a2ff5a97bf95719e38ea2e3b4105bce8  895.00   
001cca7ae9ae17fb1caed9dfb1094831 08574b074924071f4e201e151b152b4e  135.00   
                                 0da9ffd92214425d880de3f94e74ce39  112.00   
                                 21fecd254a3103704126b28478ea7980  119.00   
                                 4d7fee7877228c1497477ae53d97c214   89.00   
...                                                                   ...   
ffff564a4f9085cd26170f4732393726 cb17b798ba889575385281b7ec2fc22b  109.20   
                                 dbd024d4182504993ad1e3cd2ee9d9e9   29.40   
                                 de6517dda8e49774f58c07f80abc8d7a   69.00   
                                 e20b58fe57d487f33247e6cc1154eb9c  103.95   
                                 fa2de9e6a52f697fbd74ddbc2c15b16c  114.50   

                                                                  freight_value  \
                                                                            min   
seller_id                        product_id                                       
0015a82c2db000af6aaaf3ae2ecb0532 a2ff5a97bf95719e38ea2e3b4105bce8         21.02   
001cca7ae9ae17fb1caed9dfb1094831 08574b074924071f4e201e151b152b4e         20.65   
                                 0da9ffd92214425d880de3f94e74ce39         32.03   
                                 21fecd254a3103704126b28478ea7980         35.20   
                                 4d7fee7877228c1497477ae53d97c214         32.09   
...                                                                         ...   
ffff564a4f9085cd26170f4732393726 cb17b798ba889575385281b7ec2fc22b         13.34   
                                 dbd024d4182504993ad1e3cd2ee9d9e9         10.96   
                                 de6517dda8e49774f58c07f80abc8d7a         11.94   
                                 e20b58fe57d487f33247e6cc1154eb9c         19.32   
                                 fa2de9e6a52f697fbd74ddbc2c15b16c         12.19   

                                                                          
                                                                     max  
seller_id                        product_id                               
0015a82c2db000af6aaaf3ae2ecb0532 a2ff5a97bf95719e38ea2e3b4105bce8  21.02  
001cca7ae9ae17fb1caed9dfb1094831 08574b074924071f4e201e151b152b4e  89.82  
                                 0da9ffd92214425d880de3f94e74ce39  62.39  
                                 21fecd254a3103704126b28478ea7980  43.20  
                                 4d7fee7877228c1497477ae53d97c214  36.93  
...                                                                  ...  
ffff564a4f9085cd26170f4732393726 cb17b798ba889575385281b7ec2fc22b  13.34

In [14]:
df_test=df_order_items.groupby(['seller_id','product_id'])[['price','freight_value']].agg(['min', 'max'])
df_test.columns = ['_'.join(col) for col in df_test.columns.values]
df_test

price_min  \
seller_id                        product_id                                    
0015a82c2db000af6aaaf3ae2ecb0532 a2ff5a97bf95719e38ea2e3b4105bce8     895.00   
001cca7ae9ae17fb1caed9dfb1094831 08574b074924071f4e201e151b152b4e      69.90   
                                 0da9ffd92214425d880de3f94e74ce39      99.99   
                                 21fecd254a3103704126b28478ea7980      99.00   
                                 4d7fee7877228c1497477ae53d97c214      89.00   
...                                                                      ...   
ffff564a4f9085cd26170f4732393726 cb17b798ba889575385281b7ec2fc22b     109.20   
                                 dbd024d4182504993ad1e3cd2ee9d9e9      29.40   
                                 de6517dda8e49774f58c07f80abc8d7a      69.00   
                                 e20b58fe57d487f33247e6cc1154eb9c     103.95   
                                 fa2de9e6a52f697fbd74ddbc2c15b16c     114.50   

                                                                   price_max  \
seller_id                        product_id                                    
0015a82c2db000af6aaaf3ae2ecb0532 a2ff5a97bf95719e38ea2e3b4105bce8     895.00   
001cca7ae9ae17fb1caed9dfb1094831 08574b074924071f4e201e151b152b4e     135.00   
                                 0da9ffd92214425d880de3f94e74ce39     112.00   
                                 21fecd254a3103704126b28478ea7980     119.00   
                                 4d7fee7877228c1497477ae53d97c214      89.00   
...                                                                      ...   
ffff564a4f9085cd26170f4732393726 cb17b798ba889575385281b7ec2fc22b     109.20   
                                 dbd024d4182504993ad1e3cd2ee9d9e9      29.40   
                                 de6517dda8e49774f58c07f80abc8d7a      69.00   
                                 e20b58fe57d487f33247e6cc1154eb9c     103.95   
                                 fa2de9e6a52f697fbd74ddbc2c15b16c     114.50   

                                                                   freight_value_min  \
seller_id                        product_id                                            
0015a82c2db000af6aaaf3ae2ecb0532 a2ff5a97bf95719e38ea2e3b4105bce8              21.02   
001cca7ae9ae17fb1caed9dfb1094831 08574b074924071f4e201e151b152b4e              20.65   
                                 0da9ffd92214425d880de3f94e74ce39              32.03   
                                 21fecd254a3103704126b28478ea7980              35.20   
                                 4d7fee7877228c1497477ae53d97c214              32.09   
...                                                                              ...   
ffff564a4f9085cd26170f4732393726 cb17b798ba889575385281b7ec2fc22b              13.34   
                                 dbd024d4182504993ad1e3cd2ee9d9e9              10.96   
                                 de6517dda8e49774f58c07f80abc8d7a              11.94   
                                 e20b58fe57d487f33247e6cc1154eb9c              19.32   
                                 fa2de9e6a52f697fbd74ddbc2c15b16c              12.19   

                                                                   freight_value_max  
seller_id                        product_id                                           
0015a82c2db000af6aaaf3ae2ecb0532 a2ff5a97bf95719e38ea2e3b4105bce8              21.02  
001cca7ae9ae17fb1caed9dfb1094831 08574b074924071f4e201e151b152b4e              89.82  
                                 0da9ffd92214425d880de3f94e74ce39              62.39  
                                 21fecd254a3103704126b28478ea7980              43.20  
                                 4d7fee7877228c1497477ae53d97c214              36.93  
...                                                                              ...  
ffff564a4f9085cd26170f4732393726 cb17b798ba889575385281b7ec2fc22b              13.34  
                                 dbd024d4182504993ad1e3cd

In [15]:
df_order_items.groupby(['seller_id','product_id']).agg(
    max_price=('price', max),
    mean_price=('price', min),
    sum_freight_value=('freight_value', sum),
    )

max_price  \
seller_id                        product_id                                    
0015a82c2db000af6aaaf3ae2ecb0532 a2ff5a97bf95719e38ea2e3b4105bce8     895.00   
001cca7ae9ae17fb1caed9dfb1094831 08574b074924071f4e201e151b152b4e     135.00   
                                 0da9ffd92214425d880de3f94e74ce39     112.00   
                                 21fecd254a3103704126b28478ea7980     119.00   
                                 4d7fee7877228c1497477ae53d97c214      89.00   
...                                                                      ...   
ffff564a4f9085cd26170f4732393726 cb17b798ba889575385281b7ec2fc22b     109.20   
                                 dbd024d4182504993ad1e3cd2ee9d9e9      29.40   
                                 de6517dda8e49774f58c07f80abc8d7a      69.00   
                                 e20b58fe57d487f33247e6cc1154eb9c     103.95   
                                 fa2de9e6a52f697fbd74ddbc2c15b16c     114.50   

                                                                   mean_price  \
seller_id                        product_id                                     
0015a82c2db000af6aaaf3ae2ecb0532 a2ff5a97bf95719e38ea2e3b4105bce8      895.00   
001cca7ae9ae17fb1caed9dfb1094831 08574b074924071f4e201e151b152b4e       69.90   
                                 0da9ffd92214425d880de3f94e74ce39       99.99   
                                 21fecd254a3103704126b28478ea7980       99.00   
                                 4d7fee7877228c1497477ae53d97c214       89.00   
...                                                                       ...   
ffff564a4f9085cd26170f4732393726 cb17b798ba889575385281b7ec2fc22b      109.20   
                                 dbd024d4182504993ad1e3cd2ee9d9e9       29.40   
                                 de6517dda8e49774f58c07f80abc8d7a       69.00   
                                 e20b58fe57d487f33247e6cc1154eb9c      103.95   
                                 fa2de9e6a52f697fbd74ddbc2c15b16c      114.50   

                                                                   sum_freight_value  
seller_id                        product_id                                           
0015a82c2db000af6aaaf3ae2ecb0532 a2ff5a97bf95719e38ea2e3b4105bce8              63.06  
001cca7ae9ae17fb1caed9dfb1094831 08574b074924071f4e201e151b152b4e            4460.53  
                                 0da9ffd92214425d880de3f94e74ce39             796.21  
                                 21fecd254a3103704126b28478ea7980             114.64  
                                 4d7fee7877228c1497477ae53d97c214              69.02  
...                                                                              ...  
ffff564a4f9085cd26170f4732393726 cb17b798ba889575385281b7ec2fc22b              13.34  
                                 dbd024d4182504993ad1e3cd2ee9d9e9              27.01  
                                 de6517dda8e49774f58c07f80abc8d7a              11.94  
                                 e20b58fe57d487f33247e6cc1154eb9c              19.32  
                                 fa2de9e6a52f697fbd74ddbc2c15b16c              12.19  

[34448 rows x 3 columns]

In [16]:
df_datetime=df_order_items.copy()
df_datetime['shipping_limit_date']=pd.to_datetime(df_order_items['shipping_limit_date'])
print(df_datetime.dtypes)
df_datetime.groupby([pd.Grouper(key='shipping_limit_date', freq='M'),'seller_id','product_id']).agg(
    sum_sales=('price', sum),
    mean_price=('price', np.mean),
    sum_freight_value=('freight_value', sum),
    )

order_id                       object
order_item_id                   int64
product_id                     object
seller_id                      object
shipping_limit_date    datetime64[ns]
price                         float64
freight_value                 float64
dtype: object


sum_sales  \
shipping_limit_date seller_id                        product_id                                    
2016-09-30          a425f92c199eb576938df686728acd20 f3c2d01a84c947b078e32bbef0718962      59.50   
                    ecccfa2bb93b34a3bf033cc5d1dcdc69 5a6b04657a4c5ee34285d1e4619a96b4     134.97   
2016-10-31          011b0eaba87386a2ae96a7d32bb531d1 508d48ea59be64138f0015d2cb9c75e7      99.98   
                    01cf7e3d21494c41fb86034f2e714fa1 a760ef0bb70c954d79e62ec6023d380c      69.90   
                    024b564ae893ce8e9bfa02c10a401ece d9894482fba41f536a273ba2276d951f     467.40   
...                                                                                          ...   
2018-09-30          fa1c13f2614d7b5c4749cbc52fecda94 9865c67a74684715521d1e70226cce0b     169.80   
                    ff1fb4c404b2efe68b03350a8dc24122 25e2023ed83352bde98dc1490d14c3d8      99.00   
2020-02-29          7a241947449cc45dbfda4f9d0798d9d0 282b126b2354516c5f400154398f616d      75.99   
                                                     96ea060e41bdecc64e2de00b97068975      69.99   
2020-04-30          7a241947449cc45dbfda4f9d0798d9d0 87b92e06b320e803d334ac23966c80b1     199.98   

                                                                                       mean_price  \
shipping_limit_date seller_id                        product_id                                     
2016-09-30          a425f92c199eb576938df686728acd20 f3c2d01a84c947b078e32bbef0718962       59.50   
                    ecccfa2bb93b34a3bf033cc5d1dcdc69 5a6b04657a4c5ee34285d1e4619a96b4       44.99   
2016-10-31          011b0eaba87386a2ae96a7d32bb531d1 508d48ea59be64138f0015d2cb9c75e7       49.99   
                    01cf7e3d21494c41fb86034f2e714fa1 a760ef0bb70c954d79e62ec6023d380c       69.90   
                    024b564ae893ce8e9bfa02c10a401ece d9894482fba41f536a273ba2276d951f       77.90   
...                                                                                           ...   
2018-09-30          fa1c13f2614d7b5c4749cbc52fecda94 9865c67a74684715521d1e70226cce0b      169.80   
                    ff1fb4c404b2efe68b03350a8dc24122 25e2023ed83352bde98dc1490d14c3d8       99.00   
2020-02-29          7a241947449cc45dbfda4f9d0798d9d0 282b126b2354516c5f400154398f616d       75.99   
                                                     96ea060e41bdecc64e2de00b97068975       69.99   
2020-04-30          7a241947449cc45dbfda4f9d0798d9d0 87b92e06b320e803d334ac23966c80b1       99.99   

                                                                                       sum_freight_value  
shipping_limit_date seller_id                        product_id                                           
2016-09-30          a425f92c199eb576938df686728acd20 f3c2d01a84c947b078e32bbef0718962              15.56  
                    ecccfa2bb93b34a3bf033cc5d1dcdc69 5a6b04657a4c5ee34285d1e4619a96b4               8.49  
2016-10-31          011b0eaba87386a2ae96a7d32bb531d1 508d48ea59be64138f0015d2cb9c75e7              29.18  
                    01cf7e3d21494c41fb86034f2e714fa1 a760ef0bb70c954d79e62ec6023d380c              19.26  
                    024b564ae893ce8e9bfa02c10a401ece d9894482fba41f536a273ba2276d951f             157.80  
...                                                                                                  ...  
2018-09-30          fa1c13f2614d7b5c4749cbc52fecda94 9865c67a74684715521d1e70226cce0b               8.45  
                    ff1fb4c404b2efe68b03350a8dc24122 25e2023ed83352bde98dc1490d14c3d8               8.85  
2020-02-29          7a241947449cc45dbfda4f9d0798d9d0 282b126b2354516c5f400154398f616d              14.70  
                                                     96ea060e41bdecc64e2de00b97068975              14.66  
2020-04-30          7a241947449cc45dbfda4f9d0798d9d0 87b92e06b320e803d334ac23966c80b1             122.88  

[62486 rows x 3 columns]

In [17]:
df_datetime=df_order_items.copy()
df_datetime['shipping_limit_date']=pd.to_datetime(df_order_items['shipping_limit_date'])
df_datetime.groupby(['seller_id','product_id']).agg(
    sum_sales=pd.NamedAgg('price', sum),
    mean_price=pd.NamedAgg('price', np.mean),
    sum_freight_value=pd.NamedAgg('freight_value', sum),
    num_days=pd.NamedAgg(
        column="shipping_limit_date", 
        aggfunc=lambda x: (max(x) - min(x)))
    )

sum_sales  \
seller_id                        product_id                                    
0015a82c2db000af6aaaf3ae2ecb0532 a2ff5a97bf95719e38ea2e3b4105bce8    2685.00   
001cca7ae9ae17fb1caed9dfb1094831 08574b074924071f4e201e151b152b4e   10759.30   
                                 0da9ffd92214425d880de3f94e74ce39    1819.93   
                                 21fecd254a3103704126b28478ea7980     337.00   
                                 4d7fee7877228c1497477ae53d97c214     178.00   
...                                                                      ...   
ffff564a4f9085cd26170f4732393726 cb17b798ba889575385281b7ec2fc22b     109.20   
                                 dbd024d4182504993ad1e3cd2ee9d9e9      58.80   
                                 de6517dda8e49774f58c07f80abc8d7a      69.00   
                                 e20b58fe57d487f33247e6cc1154eb9c     103.95   
                                 fa2de9e6a52f697fbd74ddbc2c15b16c     114.50   

                                                                   mean_price  \
seller_id                        product_id                                     
0015a82c2db000af6aaaf3ae2ecb0532 a2ff5a97bf95719e38ea2e3b4105bce8  895.000000   
001cca7ae9ae17fb1caed9dfb1094831 08574b074924071f4e201e151b152b4e   95.215044   
                                 0da9ffd92214425d880de3f94e74ce39  107.054706   
                                 21fecd254a3103704126b28478ea7980  112.333333   
                                 4d7fee7877228c1497477ae53d97c214   89.000000   
...                                                                       ...   
ffff564a4f9085cd26170f4732393726 cb17b798ba889575385281b7ec2fc22b  109.200000   
                                 dbd024d4182504993ad1e3cd2ee9d9e9   29.400000   
                                 de6517dda8e49774f58c07f80abc8d7a   69.000000   
                                 e20b58fe57d487f33247e6cc1154eb9c  103.950000   
                                 fa2de9e6a52f697fbd74ddbc2c15b16c  114.500000   

                                                                   sum_freight_value  \
seller_id                        product_id                                            
0015a82c2db000af6aaaf3ae2ecb0532 a2ff5a97bf95719e38ea2e3b4105bce8              63.06   
001cca7ae9ae17fb1caed9dfb1094831 08574b074924071f4e201e151b152b4e            4460.53   
                                 0da9ffd92214425d880de3f94e74ce39             796.21   
                                 21fecd254a3103704126b28478ea7980             114.64   
                                 4d7fee7877228c1497477ae53d97c214              69.02   
...                                                                              ...   
ffff564a4f9085cd26170f4732393726 cb17b798ba889575385281b7ec2fc22b              13.34   
                                 dbd024d4182504993ad1e3cd2ee9d9e9              27.01   
                                 de6517dda8e49774f58c07f80abc8d7a              11.94   
                                 e20b58fe57d487f33247e6cc1154eb9c              19.32   
                                 fa2de9e6a52f697fbd74ddbc2c15b16c              12.19   

                                                                           num_days  
seller_id                        product_id                                          
0015a82c2db000af6aaaf3ae2ecb0532 a2ff5a97bf95719e38ea2e3b4105bce8  12 days 01:32:04  
001cca7ae9ae17fb1caed9dfb1094831 08574b074924071f4e201e151b152b4e 386 days 03:28:21  
                                 0da9ffd92214425d880de3f94e74ce39 386 days 14:50:07  
                                 21fecd254a3103704126b28478ea7980 198 days 22:45:03  
                                 4d7fee7877228c1497477ae53d97c214   9 days 05:04:53  
...                                                                             ...  
ffff564a4f9085cd26170f4732393726 cb17b798ba889575385281b7ec2fc22b   0 days 00:00:00  
                                 dbd024d4182504993ad1

# Merge

In [18]:
df_orders.merge(df_customers,
                how='left',
                left_on='customer_id',
                right_on='customer_id',
)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,barreiras,BA
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP
...,...,...,...,...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00,6359f309b166b0196dbf7ad2ac62bb5a,12209,sao jose dos campos,SP
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,da62f9e57a76d978d02ab5362c509660,11722,praia grande,SP
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,737520a9aad80b3fbbdad19b66b37b30,45920,nova vicosa,BA
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,5097a5312c8b157bb7be58ae360ef43c,28685,japuiba,RJ


In [19]:
df_order_items.merge(
    df_orders.merge(df_customers,
                    how='left',
                    left_on='customer_id',
                    right_on='customer_id',
    ),
    how='left',
    left_on='order_id',
    right_on='order_id',
).merge(df_products,
        how='left',
        left_on='product_id',
        right_on='product_id',        
).merge(df_sellers,
        how='left',
        left_on='seller_id',
        right_on='seller_id',  
       )

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,...,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,...,58.0,598.0,4.0,650.0,28.0,9.0,14.0,27277,volta redonda,SP
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,...,56.0,239.0,2.0,30000.0,50.0,30.0,40.0,3471,sao paulo,SP
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,...,59.0,695.0,2.0,3050.0,33.0,13.0,33.0,37564,borda da mata,MG
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,...,42.0,480.0,1.0,200.0,16.0,10.0,15.0,14403,franca,SP
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,...,59.0,409.0,1.0,3750.0,35.0,40.0,30.0,87900,loanda,PR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,b51593916b4b8e0d6f66f2ae24f2673d,delivered,2018-04-23 13:57:06,...,43.0,1002.0,3.0,10150.0,89.0,15.0,40.0,88303,itajai,SC
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53,84c5d4fbaf120aae381fad077416eaa0,delivered,2018-07-14 10:26:46,...,31.0,232.0,1.0,8950.0,45.0,26.0,38.0,1206,sao paulo,SP
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95,29309aa813182aaddc9b259e31b870e6,delivered,2017-10-23 17:07:56,...,43.0,869.0,1.0,967.0,21.0,24.0,19.0,80610,curitiba,PR
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72,b5e6afd5a41800fdf401e0272ca74655,delivered,2017-08-14 23:02:59,...,56.0,1306.0,1.0,100.0,20.0,20.0,20.0,4733,sao paulo,SP


# Window function